In [1]:
import sys
sys.path.append('..')
sys.path.append('../src')
sys.path.append('../data')

%load_ext autoreload

In [2]:
import json
from tqdm import tqdm

# Generate a standard dataset to evaluate

In [7]:
import random
random.seed(42)
data = json.load(open("../data/counterfact.json"))
final_data = []
# shuffle data
random.shuffle(data)
for i, d in tqdm(enumerate(data), total=len(data)):
    target_new = " " + d["requested_rewrite"]["target_new"]["str"]
    target_true = " " + d["requested_rewrite"]["target_true"]["str"]
        
    for p in d["attribute_prompts"]:
        template = "{}: " + p + "{}" + ". " + p
        final_data.append(
            {
            "base_prompt": p,
            "template": template,
            "true": target_new,
            "false": target_true,
            }
        )
        final_data[-1]["prompt"] = template.format("Redefine", final_data[-1]["false"])
    for p in d["neighborhood_prompts"]:
        template = "{}: " + p + "{}" + ". " + p
        final_data.append(
            {
            "base_prompt": p,
            "template": template,
            "true": target_true,
            "false": target_new,
            }
        )
        final_data[-1]["prompt"] = template.format("Redefine", final_data[-1]["false"])
        
unique_strs = set(json.dumps(d) for d in final_data)
final_data = [json.loads(d) for d in unique_strs]
print(len(final_data))
        

100%|██████████| 21919/21919 [00:00<00:00, 57692.49it/s]


284753


In [8]:
json.dump(final_data, open("../data/full_data.json", "w"), indent=2)

## Load HF model

In [12]:
from src.score_models import MyDataset, EvaluateMechanism
from transformers import AutoTokenizer, AutoModelForCausalLM
%load_ext autoreload
%autoreload 2

tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelForCausalLM.from_pretrained("gpt2")

dataset = MyDataset("../data/full_data.json", tokenizer=tokenizer, slice=10000)

evaluator = EvaluateMechanism("gpt2", dataset)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Tokenizing prompts: 100%|██████████| 10000/10000 [00:00<00:00, 14241.16it/s]


In [13]:
evaluator.evaluate(18)

  0%|          | 0/44 [00:00<?, ?it/s]

100%|██████████| 44/44 [00:20<00:00,  2.17it/s]


(13, 1699, 29)

In [37]:
import torch
dataset.set_len(134)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=4, shuffle=False)

[16849] [3340]
tensor([16849]) tensor([3340])
tensor([[16849,  3340]])


torch.Size([1, 2])

In [38]:
batch = next(iter(dataloader))

In [28]:
batch["target"]

tensor([16849])

In [7]:
for batch in dataloader:
    print(batch["input_ids"].shape)